# [All you need to know about skip connections](https://www.analyticsvidhya.com/blog/2021/08/all-you-need-to-know-about-skip-connections/)
Skip connections allow us to overcome the degradation problem/vanishing gradients problem where deep networks begin to perform worse than shallow networks. As the network gets deeper it become hard (read, impossible) for the weights to be propagated back to the earlier layers. Skip connections allow us to pass these weights forward from earlier layers to later layers allowing more fine grained detail to remain in the network at the end alongside more abstracted features.

## ResNet - A residual block
First we implement a residual block using skip connections in PyTorch.

In [ ]:
# import required libraries
import torch
from torch import nn
import torch.nn.functional as F
import torchvision

# basic resdidual block of ResNet
# This is generic in the sense, it could be used for downsampling of features.
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=[1, 1], downsample=None):
        """
        A basic residual block of ResNet
        Parameters
        ----------
            in_channels: Number of channels that the input have
            out_channels: Number of channels that the output have
            stride: strides in convolutional layers
            downsample: A callable to be applied before addition of residual mapping
        """
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=3, stride=stride[0], 
            padding=1, bias=False
        )

        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, stride=stride[1], 
            padding=1, bias=False
        )

        self.bn = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        # applying a downsample function before adding it to the output
        if(self.downsample is not None):
            residual = downsample(residual)

        out = F.relu(self.bn(self.conv1(x)))
        
        out = self.bn(self.conv2(out))
        # note that adding residual before activation 
        out = out + residual
        out = F.relu(out)
        return out

In [7]:
# downsample using 1*1 convolution
downsample = nn.Sequential(
    nn.Conv2d(64, 128, kernel_size=1, stride=2, bias=False),
    nn.BatchNorm2d(128)
)
# First 5 layers of ResNet34
resnet_blocks = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
    nn.MaxPool2d(kernel_size=2, stride=2),
    ResidualBlock(64,64),
    ResidualBlock(64,64),
    ResidualBlock(64,128, stride=[2,1], downsample=downsample)
)

# checking the shape
inputs = torch.rand(1,3,100,100) # single 100*100 colour image
outputs = resnet_blocks(inputs)
print(outputs.shape) # shape would be (1,128,13,13)

torch.Size([1, 128, 13, 13])


In [ ]:
# one could also use pretrained weight of ResNet trained on ImageNet
resnet34 = torchvision.models.resnet34(pretrained=True)